Machine Learning Project
Author: Nick Elias
GitHub Project Repository: https://github.com/NickElias01/datafun-07-ml 

In [2]:
import numpy as np
import pandas as pd
import matplotlib as plt
import seaborn as sns
import scipy

## Part 1 - Chart a Straight Line

"In this section, we’ll use a technique called simple linear regression to make predictions from time series data. We’ll use the 1895 through 2018 January average high temperatures in New York City to predict future average January high temperatures and to estimate the average January high temperatures for years preceding 1895." -Deitel and Deitel

In [ ]:
c = lambda f: 5 / 9 * (f-32)
temps = [(f, c(f)) for f in range(0, 101, 10)]

temps_df = pd.DataFrame(temps, columns= ['Fahrenheit', 'Celsius'])

axes = temps_df.plot(x='Fahrenheit', y='Celsius', style='.-')
y_label = axes.set_ylabel('Celsius')